# Introduction

K-mean is an iterative clustering analysis algorithm. According to the similarity principle, data objects with high similarity are divided into the same cluster, and data objects with high dissimilarity are divided into different clusters.

The main steps of K-mean are:

Step1: Suppose we want to cluster N sample observations, requiring clustering into K categories, first select K points as initial center points.

Step2: Next, according to the principle of the smallest distance from the initial center point, all observations are divided into the class where each center point is located.

Step3: There are several observations in each class, and the mean of all sample points in K classes is calculated as the K center points of the second iteration.

Step4: Repeat steps 2 and 3 according to this center until convergence (the center point no longer changes or reaches the specified number of iterations), and the clustering process ends.

# Objectives

## Part 1

### Import the MNIST dataset

For this project, we will be using the MNIST dataset. The MNIST database is a collection of 60,000 images of handwritten digits from 0 through 9, with numerical labels.

In [633]:
from sklearn.cluster import KMeans

In [634]:
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [635]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

In [636]:
from sklearn.model_selection import train_test_split

In [637]:
from pathlib import Path
import requests
import numpy as np
import gzip

mnist_url = "http://yann.lecun.com/exdb/mnist/"
img_file = "train-images-idx3-ubyte.gz"
labels_file = "train-labels-idx1-ubyte.gz"

for fname in [img_file, labels_file]:
    if Path(fname).is_file() :
        print(f"Found: {fname}")
        continue
    print(f"Downloading: {fname}")
    r = requests.get(mnist_url + fname)
    with open(fname, 'wb') as foo:
        foo.write(r.content)

with gzip.open(img_file, 'rb') as foo:
    f = foo.read()
images = np.array([b for b in f[16:]]).reshape(-1, 28*28)

with gzip.open(labels_file, 'rb') as foo:
    f = foo.read()
labels = np.array([b for b in f[8:]])

Found: train-images-idx3-ubyte.gz
Found: train-labels-idx1-ubyte.gz


images is a two-dimensional array, which contains 60,000 pieces of data

In [638]:
images.shape

(60000, 784)

labels is a one-dimensional array and corresponds to each row of images

In [639]:
labels.shape

(60000,)

Reduce the size of images and labels, which allows us to make kmean predictions faster

In [658]:
images = images[:5000]
labels = labels[:5000]

### Applying K-means Clustering

Use the k-means algorithm to split MNIST images into 10 clusters.

In [659]:
km  = KMeans(n_clusters=10)
km.fit(images)

KMeans(n_clusters=10)

Display centroids of the clusters. The centroids of the clusters consist of floating point numbers and the centroids of the clusters do not resemble numbers.

In [660]:
center = km.cluster_centers_
center.dtype

dtype('float64')

In [710]:
center = km.cluster_centers_
img = center[2].reshape(28, 28)
with np.printoptions(linewidth=5*28):
    print(img)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.0

We can find the labels of each input that is generated from K means model.

In [666]:
a=km.labels_
a

array([3, 2, 6, ..., 9, 4, 9], dtype=int32)

But these are not the true labels corresponding to each image. km.labels_ is just the group identity used for the cluster.


In order to match it with the corresponding label, I can correspond to the frequency of each label in "labels" according to the frequency of each label in "km.label_"

### Assigning Cluster Labels

In [674]:
ckmlabels=dict()
for i in a:
    if i in ckmlabels:
        ckmlabels[i]+=1
    else:
        ckmlabels[i]=1
sortkm=sorted(ckmlabels.items(), key=lambda d:d[1], reverse = True )  
sortkm

[(4, 644),
 (8, 640),
 (3, 569),
 (7, 563),
 (1, 506),
 (6, 497),
 (0, 435),
 (2, 398),
 (9, 375),
 (5, 373)]

In [676]:
cLlabels=dict()
for i in labels:
    if i in cLlabels:
        cLlabels[i]+=1
    else:
        cLlabels[i]=1
sortLl=sorted(cLlabels.items(), key=lambda d:d[1], reverse = True )  
sortLl

[(1, 563),
 (7, 550),
 (4, 535),
 (6, 501),
 (9, 495),
 (3, 493),
 (2, 488),
 (0, 479),
 (8, 462),
 (5, 434)]

According to the frequency, we can infer the labels corresponding to "km.label_". For example, 4 in "km.label_" corresponds to 1 in "labels".

In [679]:
dictionary = {4: 1, 8: 7, 3: 4, 7: 6, 1: 9, 6: 3, 0: 2, 2: 0, 9: 8, 5: 5}

In [696]:
# Replace the elements in "k_labels" with the corresponding labels
predict =np.zeros(len(a),dtype=int)
for i in range(len(a)):
    for key in dictionary:
        if a[i]==key:
            predict[i]=dictionary[key]
predict

array([4, 0, 3, ..., 8, 1, 8])

### Evaluating Clustering Algorithm

In [693]:
# Check how accurately we can predict which image corresponds to which number
predict == labels

array([False,  True, False, ..., False,  True, False])

In [694]:
accuracy = (predictions == labels).sum()/len(labels)
accuracy

0.1376

As a result, most of the predicted labels do not match.

## Part 2a

Part 2a The goal of this part is to experiment with a possible way of making k-NN predictions faster, by reducing the amount of the training data.

### Split MNIST images into training and test data.

In [700]:
X_train, X_test, Y_train, Y_test = train_test_split(images,labels,train_size=0.8,test_size=0.2)
# the size of X_train is 4000 , X_test is 1000, Y_train is 4000, Y_test is 1000

### For each digit 0-9 select all training images corresponding to this digit

In [705]:
a0=[];a1=[];a2=[];a3=[];a4=[];a5=[];a6=[];a7=[];a8=[];a9=[]

for i in range(len(Y_train)):
    if Y_train[i] == 0:
        a0.append(i)
    elif Y_train[i] == 1:
        a1.append(i)
    elif Y_train[i] == 2:
        a2.append(i)
    elif Y_train[i] == 3:
        a3.append(i)
    elif Y_train[i] == 4:
        a4.append(i)
    elif Y_train[i] == 5:
        a5.append(i)
    elif Y_train[i] == 6:
        a6.append(i)
    elif Y_train[i] == 7:
        a7.append(i)
    elif Y_train[i] == 8:
        a8.append(i)
    elif Y_train[i] == 9:
        a9.append(i)
print(len(a0),len(a1),len(a2),len(a3),len(a4),len(a5))

396 450 388 384 435 339


In [706]:
lists0 = [[] for i in range(len(a0))]
lists1 = [[] for i in range(len(a1))]
lists2 = [[] for i in range(len(a2))]
lists3 = [[] for i in range(len(a3))]
lists4 = [[] for i in range(len(a4))]
lists5 = [[] for i in range(len(a5))]
lists6 = [[] for i in range(len(a6))]
lists7 = [[] for i in range(len(a7))]
lists8 = [[] for i in range(len(a8))]
lists9 = [[] for i in range(len(a9))]

for i in range(len(a0)):
    lists0[i].append(X_train[a0[i]])
arr0 = np.array(lists0).reshape(-1,784)

for i in range(len(a1)):
    lists1[i].append(X_train[a1[i]])
arr1 = np.array(lists1).reshape(-1,784)

for i in range(len(a2)):
    lists2[i].append(X_train[a2[i]])
arr2 = np.array(lists2).reshape(-1,784)

for i in range(len(a3)):
    lists3[i].append(X_train[a3[i]])
arr3 = np.array(lists3).reshape(-1,784)

for i in range(len(a4)):
    lists4[i].append(X_train[a4[i]])
arr4 = np.array(lists4).reshape(-1,784)

for i in range(len(a5)):
    lists5[i].append(X_train[a5[i]])
arr5 = np.array(lists5).reshape(-1,784)

for i in range(len(a6)):
    lists6[i].append(X_train[a6[i]])
arr6 = np.array(lists6).reshape(-1,784)

for i in range(len(a7)):
    lists7[i].append(X_train[a7[i]])
arr7 = np.array(lists7).reshape(-1,784)

for i in range(len(a8)):
    lists8[i].append(X_train[a8[i]])
arr8 = np.array(lists8).reshape(-1,784)

for i in range(len(a9)):
    lists9[i].append(X_train[a9[i]])
arr9 = np.array(lists9).reshape(-1,784)

MNIST contains images that are 28 by 28 pixels; as a result, they will have a length of 784 once we reshape them into a 1-dimensional array.

In [712]:
img = arr0[2].reshape(28, 28)
with np.printoptions(linewidth=5*28):
    print(img)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 136 167  26   0  32 233 200  75   8   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  92 247 254 231  48  55 254 254 254  81   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 190 254 254 254 128  31 228 254 254 242  58   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  27 229 254 254 254  21   0  77 233 254 254 188  14   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   2 157 254 254 254 227  15   0   0  33  90 239 25

### Use k-means to split these images into 100 clusters.

In [714]:
km0 = KMeans(n_clusters = 100)
km0.fit(arr0)
k0=km0.cluster_centers_

km1 = KMeans(n_clusters = 100)
km1.fit(arr1)
k1=km1.cluster_centers_

km2 = KMeans(n_clusters = 100)
km2.fit(arr2)
k2=km2.cluster_centers_

km3 = KMeans(n_clusters = 100)
km3.fit(arr3)
k3=km3.cluster_centers_

km4 = KMeans(n_clusters = 100)
km4.fit(arr4)
k4=km4.cluster_centers_

km5 = KMeans(n_clusters = 100)
km5.fit(arr5)
k5=km5.cluster_centers_

km6 = KMeans(n_clusters = 100)
km6.fit(arr6)
k6=km6.cluster_centers_

km7 = KMeans(n_clusters = 100)
km7.fit(arr7)
k7=km7.cluster_centers_

km8 = KMeans(n_clusters = 100)
km8.fit(arr8)
k8=km8.cluster_centers_

km9 = KMeans(n_clusters = 100)
km9.fit(arr9)
k9=km9.cluster_centers_

I get 1000 centroids of these clusters

In [718]:
k9.shape

(100, 784)

In [734]:
Newdata = np.zeros((1000, 784))
NewdataLabel = np.zeros(1000,dtype=int)
NewdataLabel[:100] = 0
NewdataLabel[100:200] = 1;NewdataLabel[200:300] = 2;NewdataLabel[300:400] = 3;NewdataLabel[400:500] = 4;
NewdataLabel[500:600] = 5;NewdataLabel[600:700] = 6;NewdataLabel[700:800] = 7;NewdataLabel[800:900] = 8
NewdataLabel[900:1000] = 9
Newdata[:100, :] = k0
Newdata[100:200, :] = k1
Newdata[200:300, :] = k2
Newdata[300:400, :] = k3
Newdata[400:500, :] = k4
Newdata[500:600, :] = k5
Newdata[600:700, :] = k6
Newdata[700:800, :] = k7
Newdata[800:900, :] = k8
Newdata[900:1000, :] = k9


This will make the amount of training data smaller, since every cluster of the original training data will be replaced by a single centroid. There are 4000 original training data, and only 1000 new training data

## Investigate the prediction accuracy and speed of k-NN using this new training data

### Make predictions on new datasets with k-NN

In [761]:
# choose 5 neighbors


knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(Newdata,NewdataLabel)

KNeighborsClassifier(n_neighbors=10)

In [762]:
predictions = knn.predict(Newdata)
predictions == NewdataLabel

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,

In [763]:
accuracy = (predictions == NewdataLabel).sum()/len(NewdataLabel)
accuracy

0.908

As a result, some predicted labels do not match, but most of the time, k-NN can correctly predict each image.

In [764]:
%%timeit -n5 -r1
x = range(10)
for i in x:
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(Newdata,NewdataLabel)

150 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


When using new training data, knn runs at a speed of 150ms 

### Compared to the case of using unclustered training data

In [765]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,Y_train)

KNeighborsClassifier(n_neighbors=10)

In [754]:
predictions = knn.predict(X_train)

In [755]:
accuracy = (predictions == Y_train).sum()/len(Y_train)
accuracy

0.9395

The accuracy of using unclustered training data is higher than that of using new training data.

In [766]:
%%timeit -n5 -r1
x = range(10)
for i in x:
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(X_train,Y_train)

1.36 s ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


When using unclustered training dat, knn runs at a speed of 1.36s, which is slower than using new training data.

### Compared to the case of using the same amount of randomly selected training data

In [769]:
#Create the same amount of randomly selected training data, the size of X_train is 1000
X_train, X_test, Y_train, Y_test = train_test_split(images, labels, train_size=0.2, random_state=22)

In [770]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,Y_train)

KNeighborsClassifier(n_neighbors=10)

In [771]:
predictions = knn.predict(X_train)

In [772]:
accuracy = (predictions == Y_train).sum()/len(Y_train)
accuracy

0.887

The accuracy of using the same amount of randomly selected training data is lower than that of using new training data.

In [778]:
%%timeit -n5 -r1
x = range(10)
for i in x:
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(X_train,Y_train)

174 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


When using the same amount of randomly selected training data, knn runs at a speed of 174ms, which is slower than using new training data.